## Setup

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('../.env')

True

In [25]:
import pandas as pd
import psycopg2
from contextlib import closing
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter

In [3]:
EMBEDDINGS = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Connecting to pgsql database

In [32]:
db_username = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Manually construct the connection string
CONNECTION_STRING = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

params = {'user':db_username,
        'password':db_password,
        'host':db_host,
        'port':db_port,
        'database':db_name}

In [5]:
with closing(psycopg2.connect(**params)) as conn:
    with closing(conn.cursor()) as cursor:
        cursor.execute('''DROP TABLE IF EXISTS books_embedding_work_queue CASCADE''')
        cursor.execute('''DROP TABLE IF EXISTS langchain_pg_collection CASCADE''')
        cursor.execute('''DROP TABLE IF EXISTS langchain_pg_embedding CASCADE''')

## Split descriptions into smaller chunks while preserving associated metadata

In [14]:
df = pd.read_sql(con=CONNECTION_STRING, sql="SELECT book_id, description FROM books LIMIT 1000")

In [15]:
df.tail()

,book_id,description
995,33546595,Izdatel'stvo <>prodolzhaet seriiu <>novoi knig...
996,12175953,The 1956 wedding of Marilyn Monroe and Arthur ...
997,5054534,
998,18635395,"In this comprehensive and engaging volume, med..."
999,13482931,"Katie Piper is een mooie, jonge vrouw met een ..."


In [16]:
text_splitter = CharacterTextSplitter(
        separator="\n\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False
    )

In [17]:
new_list = []
for i in range(len(df.index)):
    txt = df['description'][i]
    split_text = text_splitter.split_text(txt)
    for j in range(len(split_text)):
        new_list.append([df['book_id'][i], split_text[j]])

In [18]:
df_new = pd.DataFrame(new_list, columns=df.columns)
df_new.tail()

,book_id,description
856,28276112,"1864, Washington City. One has to be careful w..."
857,33546595,Izdatel'stvo <>prodolzhaet seriiu <>novoi knig...
858,12175953,The 1956 wedding of Marilyn Monroe and Arthur ...
859,18635395,"In this comprehensive and engaging volume, med..."
860,13482931,"Katie Piper is een mooie, jonge vrouw met een ..."


## Insert embeddings into PostgrsSQL and pgvector

In [19]:
loader = DataFrameLoader(df_new, page_content_column='description')
docs = loader.load()

In [20]:
db = PGVector.from_documents(
    documents=docs,
    embedding = EMBEDDINGS,
    collection_name = "books_embeddings",
    distance_strategy = DistanceStrategy.COSINE,
    connection_string=CONNECTION_STRING
)

## Query embeddedings

In [21]:
q = "World war 2"
docs = db.similarity_search(q, k=3)

In [22]:
doc = docs[0]

# Access the document's content
doc_content = doc.page_content
# Access the document's metadata object
doc_metadata = doc.metadata

print("Content:" + doc_content)
print("Document title:" + str(doc_metadata['book_id']))

Content:The Second World War was Hitler's personal war in many senses. He intended it, prepared for it, chose the moment for launching it, planned its course, and, on several occasions between 1939 and 1942, claimed to have won it.
Although the aims he sought to achieve were old nationalist aspirations, the fact that the policy and strategy for their realization were imposed so completely by Hitler meant that if victory had come, it would have been very much a personal triumph: the ultimate failure was thus a personal one too.
This book presents all of Hitler's directives, from preparations for the invasion of Poland (31 August 1939) to his last desperate order to his troops on the Eastern Front (15 April 1945), whom he urges to choke the Bolshevik assault 'in a bath of blood'. They provide a fascinating insight into Hitler's mind and how he interpreted and reacted to events as they unfolded. The book also has detailed notes which link the Fuhrer's orders and explain the consequences o

## Query embeddings - with sql

We first need to enable the pgvector extension in our database

In [33]:
with closing(psycopg2.connect(**params)) as conn:
    with closing(conn.cursor()) as cursor:
        cursor.execute('CREATE EXTENSION IF NOT EXISTS vector;')

In [34]:
q_vector = EMBEDDINGS.embed_query("World war 2")

In [39]:
query = f"""
    SELECT cmetadata, document
    FROM public.langchain_pg_embedding
    ORDER BY embedding <-> '{q_vector}'
    LIMIT 3;
"""
with closing(psycopg2.connect(**params)) as conn:
    with closing(conn.cursor()) as cursor:
        cursor.execute(query)
        results = cursor.fetchall()

In [41]:
doc = results[0]

# Access the document's content
doc_content = doc[1]
# Access the document's metadata object
doc_metadata = doc[0]

print("Content:" + doc_content)
print("Document title:" + str(doc_metadata['book_id']))

Content:The Second World War was Hitler's personal war in many senses. He intended it, prepared for it, chose the moment for launching it, planned its course, and, on several occasions between 1939 and 1942, claimed to have won it.
Although the aims he sought to achieve were old nationalist aspirations, the fact that the policy and strategy for their realization were imposed so completely by Hitler meant that if victory had come, it would have been very much a personal triumph: the ultimate failure was thus a personal one too.
This book presents all of Hitler's directives, from preparations for the invasion of Poland (31 August 1939) to his last desperate order to his troops on the Eastern Front (15 April 1945), whom he urges to choke the Bolshevik assault 'in a bath of blood'. They provide a fascinating insight into Hitler's mind and how he interpreted and reacted to events as they unfolded. The book also has detailed notes which link the Fuhrer's orders and explain the consequences o

Save embeddings

In [ ]:
EMBEDDINGS.